In [ ]:
pip install scikeras[tensorflow]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import sklearn.metrics as sk
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(15, input_dim=15, kernel_initializer='normal', activation='relu'))
	model.add(Dense(15, input_dim=15, kernel_initializer='normal', activation='relu'))
	model.add(Dense(2, kernel_initializer='normal'))
		# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Scores/Dati/FINALE_2.csv', sep = ',')

In [ ]:
print(len(df))

In [ ]:
x = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Scores/Dati/FINALE_X.csv', sep = ',')
x = x.drop('Unnamed: 0', axis=1)

x.head(1)

In [ ]:
y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Scores/Dati/FINALE_Y.csv', sep = ',')
y = y.drop('Unnamed: 0', axis=1)

y.head(1)
scaler2 = MinMaxScaler()
scaler2.fit_transform(y.values)

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
x = x.iloc[:, 2:]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
print(type(x_train))
y_train.isnull().values.any()

In [ ]:
model=baseline_model()
estimator = KerasRegressor(model, epochs=10, batch_size=5, verbose=1)
kfold = KFold(n_splits=10) 
results = cross_val_score(estimator, x_train, y_train, cv=kfold, verbose = 5, n_jobs=-1)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))


In [ ]:
print((results))

In [ ]:
estimator.fit(x_train, y_train)
preds = estimator.predict(x_test)


In [ ]:
r_sq = estimator.score(x_train, y_train)
print('R^2:', r_sq)

adj_r2 = 1 - (1-r_sq)*(len(y_train)-1)/(len(y_train)-x_train.shape[1]-1)
print('adjusted R^2:', adj_r2)

In [ ]:
y_pred = estimator.predict(x_test)
mse = mean_squared_error(scaler2.inverse_transform(y_test), scaler2.inverse_transform(y_pred), multioutput = "raw_values")
r2_scores = sk.r2_score(y_test, y_pred, multioutput = "raw_values")

In [ ]:
print ("r2:", r2_scores)
print ("mse: ", mse)

In [ ]:
estimator.model.save("/content/drive/MyDrive/Colab Notebooks/nn_model_ondataset2")

In [ ]:
!zip -r /content/sample_data/my_model.zip /content/sample_data/my_model

In [ ]:
from google.colab import files
files.download('/content/sample_data/my_model.zip')

In [ ]:
!kill -9 -1